<a href="https://colab.research.google.com/github/JackeYou/Enjoy_LLM/blob/main/infer_local_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git lfs install
!git clone https://huggingface.co/THUDM/chatglm-6b

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
Cloning into 'chatglm-6b'...
remote: Enumerating objects: 319, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 319 (delta 64), reused 0 (delta 0), pack-reused 214
Receiving objects: 100% (319/319), 129.43 KiB | 14.38 MiB/s, done.
Resolving deltas: 100% (189/189), done.
Filtering content: 100% (9/9), 12.80 GiB | 82.98 MiB/s, done.


In [2]:
%pwd
!git clone https://github.com/JackeYou/ChatGLM-Tuning.git
%cd  ChatGLM-Tuning
!pip install -r requirements.txt

Cloning into 'ChatGLM-Tuning'...
remote: Enumerating objects: 146, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 146 (delta 46), reused 33 (delta 33), pack-reused 70
Receiving objects: 100% (146/146), 8.02 MiB | 11.39 MiB/s, done.
Resolving deltas: 100% (76/76), done.
/content/ChatGLM-Tuning
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-80_bikfe
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-80_bikfe
  Resolved https://github.com/huggingface/peft.git to commit deff03f2c251534fffd2511fc2d440e84cc54b1b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [1]:
%pwd
%cd ChatGLM-Tuning/

/content/ChatGLM-Tuning


In [2]:
import sys

sys.path.append("../")


In [7]:
from modeling_chatglm import ChatGLMForConditionalGeneration
import torch
from peft import PeftModel
from tokenization_chatglm import ChatGLMTokenizer
from transformers import AutoTokenizer
import torch.nn as nn

# 加载本地预训练模型
BASE_MODEL_DIR = "/content/chatglm-6b"
# torch.set_default_tensor_type(torch.bfloat16)
# torch.set_default_tensor_type(torch.HalfTensor)
# cpu_model = ChatGLMForConditionalGeneration.from_pretrained(BASE_MODEL_DIR).to(torch.float32)

cpu_model = ChatGLMForConditionalGeneration.from_pretrained(BASE_MODEL_DIR).float()
tokenizer = ChatGLMTokenizer.from_pretrained(BASE_MODEL_DIR)
# tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [8]:
from cover_alpaca2jsonl import format_example

instructions = [
 {'instruction': '请给出十字路口交通指示灯的三种颜色',
  'input': '',
  'output': '红色、绿色、黄色。',
 },
 {'instruction': '请给出以下资讯3个关键词',
  'input': '猪肉概念股大幅拉升，巨星农牧涨停，新五丰连续两日大涨',
  'output': '猪肉概念股，巨星农牧，新五丰。',
 },
 {'instruction': '以下哪家公司市值最高',
  'input': '腾讯、京东、淘宝',
  'output': '腾讯公司市值最高。截止2021年4月21日，腾讯公司的市值约为6.78万亿人民币，京东的市值约为1.26万亿人民币，淘宝的市值约为1.8万亿人民币。',
 },
 {'instruction': '请给出以下资讯3个关键词',
  'input': '隆基绿能董事长：光伏产品短期价格涨跌意义不大，长期看成本一定会越来越低',
  'output': '1.隆基绿能，2.光伏产品，3.成本降低',
 }
]
with torch.no_grad():
    for idx, item in enumerate(instructions):
        feature = format_example(item)
        input_text = feature['context']
        ids = tokenizer.encode(input_text)
        print(ids)
        input_ids = torch.tensor([ids])
        print(input_ids)
        out = cpu_model.generate(
            input_ids=input_ids,
            max_length=150,
            do_sample=False,
            temperature=0
        )
        out_text = tokenizer.decode(out[0])
        answer = out_text.replace(input_text, "").replace("\nEND", "").strip()
        item['infer_answer'] = answer
        print(out_text)
        print(f"chatGPT### {idx+1}.Answer:\n", item.get('output'), '\n\n')

[57010, 20012, 91492, 91432, 113999, 84994, 124643, 83825, 88947, 85107, 20004, 33049, 20012, 150001, 150004]
tensor([[ 57010,  20012,  91492,  91432, 113999,  84994, 124643,  83825,  88947,
          85107,  20004,  33049,  20012, 150001, 150004]])


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 21>:29                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/torch/utils/_contextlib.py:115 in decorate_context        │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1406 in generate         │
│                                                                                                  │
│   1403 │   │   │   │   )                                                                         │
│   1404 │   │   │                                                                                 │
│   1405 │   │   │   # 11. run greedy search                                                       │
│ ❱ 1406 │   │   │   return self.greedy_search(                                                    │
│   1407 │   │   │   │   input_ids,                                                                │
│   1408 │   │   │   │   logits_processor=logits_processor,                                        │
│   1409 │   │   │   │   stopping_criteria=stopping_criteria,                                      │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:2201 in greedy_search    │
│                                                                                                  │
│   2198 │   │   │   model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)  │
│   2199 │   │   │                                                                                 │
│   2200 │   │   │   # forward pass to get next token                                              │
│ ❱ 2201 │   │   │   outputs = self(                                                               │
│   2202 │   │   │   │   **model_inputs,                                                           │
│   2203 │   │   │   │   return_dict=True,                                                         │
│   2204 │   │   │   │   output_attentions=output_attentions,                                      │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/torch/nn/modules/module.py:1501 in _call_impl             │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used    

In [ ]:
# gpu 本地部署

In [ ]:
from modeling_chatglm import ChatGLMForConditionalGeneration
import torch
from peft import PeftModel
from tokenization_chatglm import ChatGLMTokenizer
from transformers import AutoTokenizer

# 加载本地预训练模型
torch.set_default_tensor_type(torch.cuda.HalfTensor)
BASE_MODEL_DIR = "/content/chatglm-6b"
gpu_model = ChatGLMForConditionalGeneration.from_pretrained(BASE_MODEL_DIR).half().cuda()
tokenizer = ChatGLMTokenizer.from_pretrained(BASE_MODEL_DIR)



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), Po

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 13>:13                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: must be real number, not dict

In [ ]:


from cover_alpaca2jsonl import format_example

instructions = [
 {'instruction': '请给出十字路口交通指示灯的三种颜色',
  'input': '',
  'output': '红色、绿色、黄色。',
 },
 {'instruction': '请给出以下资讯3个关键词',
  'input': '猪肉概念股大幅拉升，巨星农牧涨停，新五丰连续两日大涨',
  'output': '猪肉概念股，巨星农牧，新五丰。',
 },
 {'instruction': '以下哪家公司市值最高',
  'input': '腾讯、京东、淘宝',
  'output': '腾讯公司市值最高。截止2021年4月21日，腾讯公司的市值约为6.78万亿人民币，京东的市值约为1.26万亿人民币，淘宝的市值约为1.8万亿人民币。',
 },
 {'instruction': '请给出以下资讯3个关键词',
  'input': '隆基绿能董事长：光伏产品短期价格涨跌意义不大，长期看成本一定会越来越低',
  'output': '1.隆基绿能，2.光伏产品，3.成本降低',
 }
]
with torch.no_grad():
    for idx, item in enumerate(instructions):
        feature = format_example(item)
        input_text = feature['context']
        ids = tokenizer.encode(input_text)
        input_ids = torch.tensor([ids])
        out = gpu_model.generate(
            input_ids=input_ids,
            max_length=150,
            do_sample=False,
            temperature=0
        )
        out_text = tokenizer.decode(out[0])
        answer = out_text.replace(input_text, "").replace("\nEND", "").strip()
        item['infer_answer'] = answer
        print(out_text)
        print(f"chatGPT### {idx+1}.Answer:\n", item.get('output'), '\n\n')

Instruction: 请给出十字路口交通指示灯的三种颜色
Answer: 抱歉,作为一个人工智能助手,我没有访问互联网的能力,无法了解时事新闻,因此我无法确定您所提到的具体内容是什么。如果您能提供更多信息,我会尽力回答您的问题。
chatGPT### 1.Answer:
 红色、绿色、黄色。 


Instruction: 请给出以下资讯3个关键词
Input: 猪肉概念股大幅拉升,巨星农牧涨停,新五丰连续两日大涨
Answer: 抱歉,您的问题不完整,我无法回答。请提供更多信息或具体问题,以便更好地为您提供帮助。
chatGPT### 2.Answer:
 猪肉概念股，巨星农牧，新五丰。 


Instruction: 以下哪家公司市值最高
Input: 腾讯、京东、淘宝
Answer: 市值最高的科技公司是苹果(Apple)和亚马逊(Amazon)。根据2021年2月19日的数据,苹果的市值为6800亿美元,而亚马逊的市值为7600亿美元。
chatGPT### 3.Answer:
 腾讯公司市值最高。截止2021年4月21日，腾讯公司的市值约为6.78万亿人民币，京东的市值约为1.26万亿人民币，淘宝的市值约为1.8万亿人民币。 


Instruction: 请给出以下资讯3个关键词
Input: 隆基绿能董事长:光伏产品短期价格涨跌意义不大,长期看成本一定会越来越低
Answer: 光伏电池片是光伏发电系统中的核心部件之一,其性能直接影响整个光伏发电系统的效率和稳定性。为了提高光伏电池片的性能和可靠性,研究人员一直在探索新的材料和工艺。

近年来,随着纳米材料的应用和技术的发展,光伏电池片的材料选择和工艺也在不断更新和改进。纳米材料具有优异的光学性能和电学性能,可以显著提高光伏电池片的效率和稳定性。同时,新的制造工艺也在不断推出,如多晶硅和单晶硅的切割技术、电池片的封装技术等,以提高光伏电池片的性能和可靠性。

未来,随着可再生能源的发展和普及
chatGPT### 4.Answer:
 1.隆基绿能，2.光伏产品，3.成本降低 




In [6]:
del cpu_model
del tokenizer
import gc
gc.collect() 

0

In [ ]:
%pwd
%cd ChatGLM-Tuning/

/content/ChatGLM-Tuning
